# Benchmarking the MPS code

Here we benchmark our MPS reuslts (TEBD and TDVP) with respect to our existing exact diagonalaization data

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import h5py
import json
import matplotlib 
from mpl_toolkits.axes_grid1 import make_axes_locatable

import os
import sys  

from ast import literal_eval

from qutip import  *

sys.path.append('../../scripts') 

from qutip_tools import *


/Users/rasputin/Desktop/Uni/PhD/hoping_chain_tensors/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/rasputin/Desktop/Uni/PhD/hoping_chain_tensors/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [2]:
"""def get_file_names_on(param_list, route):
    name_list = os.listdir(route)
    try:
        name_list.remove('.DS_Store')
    except:
        pass
    # gets a list of the hdf5 file names with t dependent data depending on the values in param_list
    for subs in param_list:
        # getting strings with parameters that we want
        name_list = list(filter(lambda x: subs in x, name_list))

    return name_list
    """

"def get_file_names_on(param_list, route):\n    name_list = os.listdir(route)\n    try:\n        name_list.remove('.DS_Store')\n    except:\n        pass\n    # gets a list of the hdf5 file names with t dependent data depending on the values in param_list\n    for subs in param_list:\n        # getting strings with parameters that we want\n        name_list = list(filter(lambda x: subs in x, name_list))\n\n    return name_list\n    "

In [5]:
data_route = "../../data/sims/L=21/"
J=1

file_list = get_file_names_on(["bw2.0","om0.0","dd0fixed"], data_route)
file_list

['res_L21_maxtim18.0_bw2.0_k0.9817_jp1.0_t0.01_om0.0_dd0fixed.hdf5',
 'res_L21_maxtim18.0_bw2.0_k0.9817_jp1.0_t0.8_om0.01_dd0fixed.hdf5',
 'res_L21_maxtim18.0_bw2.0_k1.3744_jp1.0_t0.5_om0.0_dd0fixed.hdf5',
 'res_L21_maxtim18.0_bw2.0_k1.0996_jp1.0_t0.1_om0.0_dd0fixed.hdf5',
 'res_L21_maxtim18.0_bw2.0_k1.4923_jp1.0_t0.05_om0.01_dd0fixed.hdf5',
 'res_L21_maxtim18.0_bw2.0_k1.4137_jp1.0_t0.2_om0.01_dd0fixed.hdf5',
 'res_L21_maxtim18.0_bw2.0_k1.5708_jp1.0_t0.6_om0.01_dd0fixed.hdf5',
 'res_L21_maxtim18.0_bw2.0_k1.5708_jp1.0_t0.3_om0.05_dd0fixed.hdf5',
 'res_L21_maxtim18.0_bw2.0_k1.1781_jp1.0_t0.5_om0.0_dd0fixed.hdf5',
 'res_L21_maxtim18.0_bw2.0_k1.4137_jp1.0_t0.4_om0.0_dd0fixed.hdf5',
 'res_L21_maxtim18.0_bw2.0_k1.2566_jp1.0_t0.01_om0.01_dd0fixed.hdf5',
 'res_L21_maxtim18.0_bw2.0_k1.5708_jp1.0_t0.1_om0.0_dd0fixed.hdf5',
 'res_L21_maxtim18.0_bw2.0_k1.5708_jp1.0_t0.6_om0.05_dd0fixed.hdf5',
 'res_L21_maxtim18.0_bw2.0_k1.5708_jp1.0_t0.3_om0.01_dd0fixed.hdf5',
 'res_L21_maxtim18.0_bw2.0_k1.4923_jp